# Back Propagation
### Regression

In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.DataFrame([[8,8,4],[7,9,5],[6,10,6],[5,12,7]], columns=['cgpa', 'profile_score', 'lpa'])
df.head()

,cgpa,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,7


In [15]:
def initialize_parameters(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)         
    for l in range(1, L):
        parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l]))*0.1
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters
    
def linear_forward(A_prev, W, b):
    Z = np.dot(W.T, A_prev) + b
    return Z
    
def L_layer_forward(X, parameters):
    A = X
    L = len(parameters) // 2  # number of layers in the neural network   
    for l in range(1, L+1):
        A_prev = A 
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        #print("A"+str(l-1)+": ", A_prev)
        #print("W"+str(l)+": ", Wl)
        #print("b"+str(l)+": ", bl)
        #print("--"*20)
        A = linear_forward(A_prev, Wl, bl)
        #print("A"+str(l)+": ", A)
        #print("**"*20)           
    return A,A_prev

def update_parameters(parameters,y,y_hat,A1,X):
    parameters['W2'][0][0] = parameters['W2'][0][0] + (0.001 * 2 * (y - y_hat)*A1[0][0])
    parameters['W2'][1][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat)*A1[1][0])
    parameters['b2'][0][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat))

    parameters['W1'][0][0] = parameters['W1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[0][0])
    parameters['W1'][0][1] = parameters['W1'][0][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0]*X[1][0])
    parameters['b1'][0][0] = parameters['b1'][0][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][0][0])

    parameters['W1'][1][0] = parameters['W1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[0][0])
    parameters['W1'][1][1] = parameters['W1'][1][1] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0]*X[1][0])
    parameters['b1'][1][0] = parameters['b1'][1][0] + (0.001 * 2 * (y - y_hat)*parameters['W2'][1][0])

In [16]:
# epochs implementation

# Parameter initialization
#here, [2,2,1]=[no of features Entry,Activation layer 1 nodes,output layer nodes]
#------------------> (since, here we have only one hidden layer = Activation layer 1)
parameters = initialize_parameters([2,2,1])
epochs = 5

for i in range(epochs):

  Loss = []

  for j in range(df.shape[0]):

    X = df[['cgpa', 'profile_score']].values[j].reshape(2,1) #Shape(no of features, no. of training example)
    y = df[['lpa']].values[j][0]

    y_hat,A1 = L_layer_forward(X,parameters) #Forward Propagation
    y_hat = y_hat[0][0] #Get y_hat value

    update_parameters(parameters,y,y_hat,A1,X) #Gradient Descent to update weights and biases

    Loss.append((y-y_hat)**2) #Calculate loss MSE

  print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean()) #Calculate Avg loss for epoch

parameters

Epoch -  1 Loss -  25.321744156025517
Epoch -  2 Loss -  18.320004165722047
Epoch -  3 Loss -  9.473661050729628
Epoch -  4 Loss -  3.2520938634031613
Epoch -  5 Loss -  1.3407132589299962


{'W1': array([[0.26507636, 0.38558861],
        [0.27800387, 0.40980287]]),
 'b1': array([[0.02749056],
        [0.02974394]]),
 'W2': array([[0.41165744],
        [0.48302736]]),
 'b2': array([[0.48646246]])}

# With Keras 

In [25]:
from tensorflow import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense

In [18]:
model=Sequential()
model.add(Dense(2,activation='linear',input_dim=2))
model.add(Dense(1,activation='linear'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.get_weights()

[array([[ 1.129933  , -0.83946884],
        [-1.1394371 , -0.83429044]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[-0.29577208],
        [ 0.5704421 ]], dtype=float32),
 array([0.], dtype=float32)]

In [21]:
model.set_weights([np.array([[0.1 ,  0.1 ],
        [ 0.1, 0.1 ]], dtype='float32'),
 np.array([0., 0.], dtype='float32'),
 np.array([[0.1],
        [0.1]], dtype='float32'),
 np.array([0.], dtype='float32')])

In [22]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss='mean_squared_error')

In [23]:
X = df[['cgpa', 'profile_score']]
y = df[['lpa']]
history=model.fit(X,y,epochs=75,verbose=1,batch_size=1)

Epoch 1/75
4/4 [==============================] - 1s 3ms/step - loss: 27.8873
Epoch 2/75
4/4 [==============================] - 0s 4ms/step - loss: 27.5833
Epoch 3/75
4/4 [==============================] - 0s 3ms/step - loss: 27.2353
Epoch 4/75
4/4 [==============================] - 0s 2ms/step - loss: 26.9247
Epoch 5/75
4/4 [==============================] - 0s 2ms/step - loss: 26.5505
Epoch 6/75
4/4 [==============================] - 0s 2ms/step - loss: 26.2174
Epoch 7/75
4/4 [==============================] - 0s 3ms/step - loss: 25.8200
Epoch 8/75
4/4 [==============================] - 0s 2ms/step - loss: 25.4600
Epoch 9/75
4/4 [==============================] - 0s 3ms/step - loss: 25.0667
Epoch 10/75
4/4 [==============================] - 0s 2ms/step - loss: 24.6302
Epoch 11/75
4/4 [==============================] - 0s 3ms/step - loss: 24.2574
Epoch 12/75
4/4 [==============================] - 0s 3ms/step - loss: 23.8000
Epoch 13/75
4/4 [==============================] - 0s 3ms/ste

In [24]:
model.get_weights()

[array([[0.37347668, 0.37347668],
        [0.36544046, 0.36544046]], dtype=float32),
 array([0.2721442, 0.2721442], dtype=float32),
 array([[0.37275404],
        [0.37275404]], dtype=float32),
 array([0.20465098], dtype=float32)]